In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
cd /content/drive/MyDrive/Herron_lab/Neural_data/Neural - MPQ

In [ ]:
import h5py as hp
import numpy as np
import pandas as pd
import os
from google.colab import auth
auth.authenticate_user()
import sklearn
import scipy
import gspread
import scipy
import re
from google.auth import default
import matplotlib.pyplot as plt
from sklearn.decomposition import FastICA
import scipy.signal as signal
import scipy.linalg as la
import tqdm
# import mne
%matplotlib inline
creds, _ = default()
gc = gspread.authorize(creds)


In [ ]:
#load the survey sheet

spreadsheet = gc.open('6c29e3_MPQ_survey_responses').sheet1
rows = spreadsheet.get_all_values()
survey_response = pd.DataFrame(rows[1:], columns=rows[0])

In [ ]:
survey_response

In [ ]:


'''
Converting pain scores into labels
Where keys are specific days and the values are the labels of each trial of the day
defaultdict(list,
            {'2': [1, 1, 1],
             '3': [1, 1, 1],
             '4': [1, 0, 0, 1], #test
             '5': [0, 0, 0],
             '6': [0],
             '7': [0, 0]})'''
from collections import defaultdict

labels=defaultdict(list)
for iter in range(len(survey_response)):
  labels[survey_response.day[iter]].append(survey_response.Total[iter])

Pain=[]
for iter in range(len(survey_response)):
  Pain.append(int(survey_response.Total[iter]))

for key,value in labels.items():

  for pains in range(len(value)):
    if int(value[pains])>np.median(Pain):
      value[pains]=1
    else:
      value[pains]=0


In [ ]:
'''Loading the dataset
'''
h5files=[]
path='/content/drive/MyDrive/Herron_lab/Neural_data/Neural - MPQ' #provide the path to SEED EEG dataset
files=os.listdir(path)
for index in range(len(files)):
  if files[index].endswith('.h5'):
    h5files.append(files[index])

In [ ]:
h5files

In [ ]:
#Calculating total power in band for the data
def bandpower(x_data, fs, fmin, fmax):
  power_in_band=[]
  for i in range(x_data.shape[0]):
    x=x_data[i,:]
    f, Pxx = scipy.signal.welch(x, fs=fs)
    ind_min = np.argmax(f > fmin) - 1
    ind_max = np.argmax(f > fmax) - 1
    power_in_band.append(np.trapz(Pxx[ind_min: ind_max], f[ind_min: ind_max]))
  return np.array(power_in_band)

#load the pain data and calculate the band power for theta, alpha, beta, gamma
def calc_features_bandpower(data):

  theta_power = bandpower(data,512,4,8)
  alpha_power = bandpower(data,512,8,13)
  beta_power = bandpower(data,512,13,30)
  gamma_power = bandpower(data,512,30,70)
  features = []
  for i in range(198):
    features.append([theta_power[i],alpha_power[i],beta_power[i],gamma_power[i]])
  return np.array(features)



In [ ]:
#PIB using butterworth and hilbert
def filtering(data_ecog):
  sos_delta=signal.butter(2,[0.1,4],btype='bandpass',analog=False, output='sos',fs=500)
  sos_theta=signal.butter(2,[4,8],btype='bandpass',analog=False, output='sos',fs=500)
  sos_alpha=signal.butter(2,[8,13],btype='bandpass',analog=False, output='sos',fs=500)
  sos_beta=signal.butter(2,[13,30],btype='bandpass',analog=False, output='sos',fs=500)
  sos_gamma=signal.butter(2,[30,60],btype='bandpass',analog=False, output='sos',fs=500)
  sos_highgamma=signal.butter(2,[60,200],btype='bandpass',analog=False, output='sos',fs=500)
  delta_filter = signal.sosfilt(sos_delta,data_ecog)
  theta_filter = signal.sosfilt(sos_theta,data_ecog)
  alpha_filter = signal.sosfilt(sos_alpha,data_ecog)
  beta_filter = signal.sosfilt(sos_beta,data_ecog)
  gamma_filter = signal.sosfilt(sos_gamma,data_ecog)
  highgamma_filter = signal.sosfilt(sos_highgamma,data_ecog)
  return [delta_filter,theta_filter,alpha_filter,beta_filter,gamma_filter,highgamma_filter]

def PIB(data):
    filter_series=filtering(data)
    power_delta = np.sum(np.abs(signal.hilbert(filter_series[0]))**2,axis=1)
    power_theta= np.sum(np.abs(signal.hilbert(filter_series[1]))**2,axis=1)
    power_alpha= np.sum(np.abs(signal.hilbert(filter_series[2]))**2,axis=1)
    power_beta= np.sum(np.abs(signal.hilbert(filter_series[3]))**2,axis=1)
    power_gamma= np.sum(np.abs(signal.hilbert(filter_series[4]))**2,axis=1)
    power_high_gamma= np.sum(np.abs(signal.hilbert(filter_series[5]))**2,axis=1)
    feature_power=[]
    for i in range(198):
      feature_power.append([power_delta[i],power_theta[i],power_alpha[i],power_beta[i],power_gamma[i],power_high_gamma[i]])
    return np.array(feature_power)


    # for filters in range(len(filter_series)):
        # hilbert_transformed_signal = signal.hilbert(filter_series[filters])
        # breakpoint()

        # power = np.abs(hilbert_transformed_signal)**2
        # ax.plot(power)



In [ ]:
def sliding_window_augmentation(data): #198x153600
  window_sizee = 15360
  stride = 15360
  final_arrays=[]
  for i in range(10):
    subarray = data[:,15360*i:15360*(i+1)]
    final_arrays.append(subarray)
  return np.array(final_arrays)





In [ ]:
# Common Spatial Pattern implementation in Python, used to build spatial filters for identifying task-related activity.

#Code obtained from github

# CSP takes any number of arguments, but each argument must be a collection of trials associated with a task
# That is, for N tasks, N arrays are passed to CSP each with dimensionality (# of trials of task N) x (feature vector)
# Trials may be of any dimension, provided that each trial for each task has the same dimensionality,
# otherwise there can be no spatial filtering since the trials cannot be compared
def CSP(*tasks):
	if len(tasks) < 2:
		print("Must have at least 2 tasks for filtering.")
		return (None,) * len(tasks)
	else:
		filters = ()
		# CSP algorithm
		# For each task x, find the mean variances Rx and not_Rx, which will be used to compute spatial filter SFx
		iterator = range(0,len(tasks))
		for x in iterator:
			# Find Rx
			Rx = covarianceMatrix(tasks[x][0])

			for t in range(1,len(tasks[x])):
				Rx += covarianceMatrix(tasks[x][t])
			Rx = Rx / len(tasks[x])

			# Find not_Rx
			count = 0
			not_Rx = Rx * 0
			for not_x in [element for element in iterator if element != x]:
				for t in range(0,len(tasks[not_x])):
					not_Rx += covarianceMatrix(tasks[not_x][t])
					count += 1
			not_Rx = not_Rx / count

			# Find the spatial filter SFx
			SFx = spatialFilter(Rx,not_Rx)
			filters += (SFx,)

			# Special case: only two tasks, no need to compute any more mean variances
			if len(tasks) == 2:
				filters += (spatialFilter(not_Rx,Rx),)
				break
		return filters

# covarianceMatrix takes a matrix A and returns the covariance matrix, scaled by the variance
def covarianceMatrix(A):
	Ca = np.dot(A,np.transpose(A))/np.trace(np.dot(A,np.transpose(A)))
	return Ca

# spatialFilter returns the spatial filter SFa for mean covariance matrices Ra and Rb
def spatialFilter(Ra,Rb):
	R = Ra + Rb
	E,U = la.eig(R)
	breakpoint()
	# CSP requires the eigenvalues E and eigenvector U be sorted in descending order
	ord = np.argsort(E)
	ord = ord[::-1] # argsort gives ascending order, flip to get descending
	E = E[ord]
	U = U[:,ord]

	# Find the whitening transformation matrix
	P = np.dot(np.sqrt(la.inv(np.diag(E))),np.transpose(U))

	# The mean covariance matrices may now be transformed
	Sa = np.dot(P,np.dot(Ra,np.transpose(P)))
	Sb = np.dot(P,np.dot(Rb,np.transpose(P)))

	# Find and sort the generalized eigenvalues and eigenvector
	E1,U1 = la.eig(Sa,Sb)
	ord1 = np.argsort(E1)
	ord1 = ord1[::-1]
	E1 = E1[ord1]
	U1 = U1[:,ord1]

	# The projection matrix (the spatial filter) may now be obtained
	SFa = np.dot(np.transpose(U1),P)
	return SFa.astype(np.float32)

In [ ]:
'''Calculate the Band features for the entire data'''
pain_data_train=[]
nopain_data_train = []
pain_data_test=[]
nopain_data_test = []
for index in tqdm.tqdm(range(len(h5files))):
# for index in range(5,6):

  dayX=h5files[index].split('_')[1] #dx where x \in {2,3,4,5,6,7}
  day_match = re.search(r'\d+', dayX)
  day = day_match.group() if day_match else None #gets the numeric value(x) in dx
  matpath=os.path.join(path,h5files[index])
  # print(matpath)
  data_h5=hp.File(matpath,'r')
  matrix = data_h5['neural_windows'][()]
  trials = matrix.shape[0]
  for j in range(trials):
    ecog_data= matrix[j]
    augmented_data=sliding_window_augmentation(ecog_data)
    for subarrays in augmented_data:
    # print(f'ecog data of trial {j}','\n',ecog_data,'\n','shape=',ecog_data.shape)
      # data=calc_features_bandpower(subarrays)
      data = PIB(subarrays)

      if day=='4':
        if labels[day][j]==0:
          nopain_data_test.append(data)
        else:
          pain_data_test.append(data)


      else:
        if labels[day][j]==0:
          nopain_data_train.append(data)
        else:
          pain_data_train.append(data)


In [ ]:

pain_data_train = np.array(pain_data_train)
nopain_data_train = np.array(nopain_data_train)

pain_data_test = np.array(pain_data_test)
nopain_data_test = np.array(nopain_data_test)


In [ ]:
pain_data_train.shape

In [ ]:
csp_train_data=[]
csp_test_data=[]
for datas in range(pain_data_train.shape[0]):
  csp = CSP(pain_data_train[datas].reshape(1,pain_data_train[datas].shape[0],pain_data_train[datas].shape[1]),nopain_data_train[datas].reshape(1,nopain_data_train[datas].shape[0],nopain_data_train[datas].shape[1]))
  csp_train_data.append(list(csp))
csp_train_data=np.array(csp_train_data)
for datas in range(pain_data_test.shape[0]):
  csp = CSP(pain_data_test[datas].reshape(1,pain_data_test[datas].shape[0],pain_data_test[datas].shape[1]),nopain_data_test[datas].reshape(1,nopain_data_test[datas].shape[0],nopain_data_test[datas].shape[1]))
  csp_test_data.append(list(csp))
csp_test_data=np.array(csp_test_data)

In [ ]:
csp_train_data.shape

In [ ]:
validation_pain_csp = csp_test_data[:,0]
validation_nopain_csp = csp_test_data[:,1]
train_pain_csp = csp_train_data[:,0]
train_nopain_csp = csp_train_data[:,1]

In [ ]:
pain_data_test.shape

In [ ]:
validation_pain=[]
validation_nopain=[]
train_pain =[]
train_nopain =[]


csp_filters_transposed_validation_pain = np.transpose(validation_pain_csp, (0, 2, 1))

# Apply CSP transformation
validation_pain = np.matmul(np.transpose(pain_data_test,(0, 2, 1)), csp_filters_transposed_validation_pain)

# Optionally, normalize the transformed data
validation_pain_normalized = validation_pain / np.linalg.norm(validation_pain, axis=(1, 2), keepdims=True)





csp_filters_transposed_validation_nopain = np.transpose(validation_nopain_csp, (0, 2, 1))

# Apply CSP transformation
validation_nopain = np.matmul(np.transpose(nopain_data_test,(0, 2, 1)), csp_filters_transposed_validation_nopain)

# Optionally, normalize the transformed data
validation_nopain_normalized = validation_nopain / np.linalg.norm(validation_nopain, axis=(1, 2), keepdims=True)





csp_filters_transposed_train_pain = np.transpose(train_pain_csp, (0, 2, 1))

# Apply CSP transformation
train_pain = np.matmul(np.transpose(pain_data_train,(0, 2, 1)), csp_filters_transposed_train_pain)

# Optionally, normalize the transformed data
train_pain_normalized = train_pain / np.linalg.norm(train_pain, axis=(1, 2), keepdims=True)




csp_filters_transposed_train_nopain = np.transpose(train_nopain_csp, (0, 2, 1))

# Apply CSP transformation
train_nopain = np.matmul(np.transpose(nopain_data_train,(0, 2, 1)), csp_filters_transposed_train_nopain)

# Optionally, normalize the transformed data
train_nopain_normalized = train_nopain / np.linalg.norm(train_nopain, axis=(1, 2), keepdims=True)

In [ ]:
train_pain.shape

In [ ]:
test_set = np.concatenate([validation_pain_normalized,validation_nopain_normalized])
train_set = np.concatenate([train_pain_normalized, train_nopain_normalized])

In [ ]:
train_set.shape

In [ ]:
y_test=np.concatenate([np.ones(validation_pain_normalized.shape[0]),np.zeros(validation_nopain_normalized.shape[0])])
y_train = np.concatenate([np.ones(train_pain_normalized.shape[0]),np.zeros(train_nopain_normalized.shape[0])])

In [ ]:
#Testing Null hypothesis (Optional)
import random
y_train_shuffled = np.array(random.sample(list(y_train), len(y_train)))

In [ ]:
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
lda = LinearDiscriminantAnalysis()

In [ ]:
train_set = train_set.reshape(train_set.shape[0],-1)

In [ ]:
train_set.shape

In [ ]:
test_set = test_set.reshape(test_set.shape[0],-1)

In [ ]:
lda.fit(train_set.reshape(train_set.shape[0],-1),y_train_shuffled)

In [ ]:
test_set.shape

In [ ]:
train_set.shape

In [ ]:
test_set.shape

In [ ]:
y_predict = lda.predict(test_set.reshape(test_set.shape[0],-1))

In [ ]:
y_test

In [ ]:
y_predict

In [ ]:
#Null hypothesis prediction
y_predict_nh = lda.predict(test_set.reshape(test_set.shape[0],-1))
acc=0
for elements in range(len(y_test)):
  if y_test[elements]==y_predict_nh[elements]:
    acc+=1
print("accuracy is ",acc/len(y_test))

In [ ]:
acc=0
for elements in range(len(y_test)):
  if y_test[elements]==y_predict[elements]:
    acc+=1
print("accuracy is ",acc/len(y_test))

Trying out with LSTM

In [ ]:
from keras.models import Sequential
from keras.layers import LSTM, Dense, Dropout, BatchNormalization
from keras.layers import Conv1D, MaxPooling1D, Activation, Flatten
from keras.utils import to_categorical
import numpy as np
import h5py
import matplotlib.pyplot as plt
from sklearn import preprocessing

In [ ]:
num_classes=2

N_train, T_train, C_train = train_set.shape
N_test, T_test, C_test = test_set.shape

# add dummy zeros for y classification, convert class vectors to binary class matrices.
y_train = to_categorical(y_train, num_classes)
y_test = to_categorical(y_test, num_classes)

In [ ]:
sampling = 1

X_train = train_set.reshape(N_train,int(T_train/sampling), sampling, C_train)[:,:,0,:]
X_test = test_set.reshape(N_test,int(T_test/sampling), sampling, C_test)[:,:,0,:]

# get new data dimension
N_train, T_train, C_train = X_train.shape
N_test, T_test, C_test = X_test.shape

print('X_train: ', X_train.shape)
print('y_train: ', y_train.shape)

print('X_test: ', X_test.shape)
print('y_test: ', y_test.shape)

In [ ]:
# Expected input batch shape: (batch_size, timesteps, data_dim)
# Note that we have to provide the full batch_input_shape since the network is stateful.
# the sample of index i in batch k is the follow-up for the sample i in batch k-1.

# perhaps should try masking layer

data_dim = C_train
timesteps = T_train
batch_size = 8
num_epoch = 20

# make a sequential model
model = Sequential()

# add 1-layer cnn
model.add(Conv1D(4, kernel_size=20, strides=4,
          input_shape=(timesteps, data_dim)))
model.add(Activation('relu'))
# model.add(Dropout(0.5))
model.add(BatchNormalization())
model.add(MaxPooling1D(pool_size=4, strides=4))


# add 2-layer lstm
model.add(LSTM(30, return_sequences=True, stateful=False))
# model.add(Dropout(0.5))
model.add(BatchNormalization())
model.add(LSTM(20, return_sequences=True, stateful=False))
# model.add(Dropout(0.5))
model.add(BatchNormalization())
model.add(Flatten())
model.add(Dense(num_classes, activation='sigmoid'))

# set loss function and optimizer
model.compile(loss='binary_crossentropy',
              optimizer='SGD',
              metrics=['accuracy'])

In [ ]:
history = model.fit(X_train, y_train,
                    batch_size=batch_size,
                    epochs=num_epoch,
                    shuffle=False,
                    validation_data=(X_test, y_test))

In [ ]:
model.evaluate(X_test,y_test,batch_size=N_test)

In [ ]:
print(history.history.keys())

# summarize history for accuracy
plt.plot(history.history['accuracy'])
plt.plot(history.history['val_accuracy'])
plt.title('model accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['train', 'val'], loc='upper left')
plt.show()

# summarize history for loss
plt.plot(history.history['loss'],'o')
plt.plot(history.history['val_loss'],'o')
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'val'], loc='upper left')
plt.show()

Visualizing the two channels of ECoG data before and after applying spatial filtering

In [ ]:
pain_after_csp = train_pain[0][:,:2].T
nopain_after_csp = train_nopain[0][:,:2].T

pain_before_csp = pain_data_train[0].T[:,:2].T
nopain_before_csp = nopain_data_train[0].T[:,:2].T

In [ ]:
import matplotlib.pyplot as plt
import math
fig, ax = plt.subplots(figsize=(10, 10))
ax.scatter(pain_before_csp[0], pain_before_csp[1], c='b', s=10.0, alpha=0.8, label="Pain data ")
ax.scatter(nopain_before_csp[0], nopain_before_csp[1], c='r', s=10.0, alpha=0.8, label="No Pain data ")

ax.grid()
ax.set_title('Before CSP')
ax.set_xlabel('Channel 1')
ax.set_ylabel('Channel 2')
ax.legend(loc = 0)
ax.set_xlim([0,1*math.pow(10,-5)])
ax.set_ylim([0,0.5*math.pow(10,-5)])

In [ ]:
import matplotlib.pyplot as plt
import math
fig, ax = plt.subplots(figsize=(10, 10))
ax.scatter(pain_after_csp[0], pain_after_csp[1], c='b', s=10.0, alpha=0.8, label="Pain data ")
ax.scatter(nopain_after_csp[0], nopain_after_csp[1], c='r', s=10.0, alpha=0.8, label="No Pain data ")

ax.grid()
ax.set_title('After CSP')
ax.set_xlabel('Channel 1')
ax.set_ylabel('Channel 2')
ax.legend(loc = 0)


# **These plots doesn't make sense. Instead try focusing on a specific band and then plot CSP plots for that band**

In [ ]:
from scipy import signal
from scipy.stats import zscore

def filtering_and_downsample(data_ecog, downsample_factor=5):
    sos_delta = signal.butter(2, [0.1, 4], btype='bandpass', analog=False, output='sos', fs=500)
    sos_theta = signal.butter(2, [4, 8], btype='bandpass', analog=False, output='sos', fs=500)
    sos_alpha = signal.butter(2, [8, 13], btype='bandpass', analog=False, output='sos', fs=500)
    sos_beta = signal.butter(2, [13, 30], btype='bandpass', analog=False, output='sos', fs=500)
    sos_gamma = signal.butter(2, [30, 60], btype='bandpass', analog=False, output='sos', fs=500)
    sos_highgamma = signal.butter(2, [60, 200], btype='bandpass', analog=False, output='sos', fs=500)

    delta_filter = signal.sosfilt(sos_delta, data_ecog)
    theta_filter = signal.sosfilt(sos_theta, data_ecog)
    alpha_filter = signal.sosfilt(sos_alpha, data_ecog)
    beta_filter = signal.sosfilt(sos_beta, data_ecog)
    gamma_filter = signal.sosfilt(sos_gamma, data_ecog)
    highgamma_filter =signal.sosfilt(sos_highgamma, data_ecog)


    # Downsample each filtered signal and zscore values

    delta_downsampled = zscore(signal.decimate(delta_filter, downsample_factor),axis=0)
    theta_downsampled = zscore(signal.decimate(theta_filter, downsample_factor),axis=0)
    alpha_downsampled = zscore(signal.decimate(alpha_filter, downsample_factor),axis=0)
    beta_downsampled = zscore(signal.decimate(beta_filter, downsample_factor),axis=0)
    gamma_downsampled = zscore(signal.decimate(gamma_filter, downsample_factor),axis=0)
    highgamma_downsampled = zscore(signal.decimate(highgamma_filter, downsample_factor),axis=0)

    return [delta_downsampled, theta_downsampled, alpha_downsampled, beta_downsampled, gamma_downsampled, highgamma_downsampled]


In [ ]:
#Load the high pain data. #day 2 trial 2
highpain_path=os.path.join(path,h5files[0])
# print(matpath)
data_h5_high=hp.File(highpain_path,'r')
highpain_data = data_h5_high['neural_windows'][()][1]

#Load the low pain data. #day 5 trial 2
lowpain_path=os.path.join(path,h5files[3])
# print(matpath)
data_h5_low=hp.File(lowpain_path,'r')
lowpain_data = data_h5_low['neural_windows'][()][1]

In [ ]:
delta_filter_high,theta_filter_high,alpha_filter_high,beta_filter_high,gamma_filter_high,highgamma_filter_high = filtering_and_downsample(highpain_data)
delta_filter_low,theta_filter_low,alpha_filter_low,beta_filter_low,gamma_filter_low,highgamma_filter_low = filtering_and_downsample(lowpain_data)

In [ ]:
delta_filter_high.shape

Let us first concentrate on delta band


focusing on the Channel 1 and Channel 2 gives this before applying CSP

In [ ]:
import matplotlib.pyplot as plt
import math
fig, ax = plt.subplots(figsize=(10, 10))
ax.scatter(delta_filter_high[0], delta_filter_high[1], c='b', s=10.0, alpha=0.8, label="Pain data ")
ax.scatter(delta_filter_low[0], delta_filter_low[1], c='r', s=10.0, alpha=0.8, label="No Pain data ")

ax.grid()
ax.set_title('Before CSP')
ax.set_xlabel('Channel 1')
ax.set_ylabel('Channel 2')
ax.legend(loc = 0)

Now let us apply CSP and then check

In [ ]:
csp=np.array(CSP(delta_filter_high.reshape(1,delta_filter_high.shape[0],delta_filter_high.shape[1]),delta_filter_low.reshape(1,delta_filter_low.shape[0],delta_filter_low.shape[1])))
pain_csp,nopain_csp = np.transpose(csp[0],(1,0)),np.transpose(csp[1],(1,0))

pain_delta_after_csp = np.matmul(np.transpose(delta_filter_high,(1,0)),pain_csp).T
nopain_delta_after_csp = np.matmul(np.transpose(delta_filter_low,(1,0)),nopain_csp).T


Now let us plot the csp values

In [ ]:
import matplotlib.pyplot as plt
import math
fig, ax = plt.subplots(figsize=(10, 10))
ax.scatter(pain_delta_after_csp[0], pain_delta_after_csp[1], c='b', s=10.0, alpha=0.8, label="Pain data ")
ax.scatter(nopain_delta_after_csp[0], nopain_delta_after_csp[1], c='r', s=10.0, alpha=0.8, label="No Pain data ")

ax.grid()
ax.set_title('After CSP')
ax.set_xlabel('Channel 1')
ax.set_ylabel('Channel 2')
ax.legend(loc = 0)

Making a generalzied version of the functions

In [ ]:
import matplotlib.pyplot as plt
import math
def plot_features(data_high,data_low,choice,chx,chy,feature_names):
  fig, ax = plt.subplots(figsize=(5,5))
  ax.scatter(data_high[chx][::10], data_high[chy][::10], c='b', s=15.0, alpha=0.4, label="Pain data ") #remove the step size
  ax.scatter(data_low[chx][::10], data_low[chy][::10], c='r', s=15.0, alpha=0.4, label="No Pain data ")#remove the step size

  ax.grid()
  ax.set_title(choice)
  ax.set_xlabel(f'Channel {feature_names[chx]}')
  ax.set_ylabel(f'Channel {feature_names[chy]}')
  ax.legend(loc = 0)


def csp_cal_appply(data_high, data_low):
  csp=np.array(CSP(data_high.reshape(1,data_high.shape[0],data_high.shape[1]),data_low.reshape(1,data_low.shape[0],data_low.shape[1])))
  pain_csp,nopain_csp = np.transpose(csp[0],(1,0)),np.transpose(csp[1],(1,0))

  pain_data_after_csp = np.matmul(np.transpose(data_high,(1,0)),pain_csp).T
  nopain_data_after_csp = np.matmul(np.transpose(data_low,(1,0)),nopain_csp).T
  return [pain_data_after_csp,nopain_data_after_csp]

Let us try with theta band

In [ ]:
nopain_bands = [delta_filter_low,theta_filter_low,alpha_filter_low,beta_filter_low,gamma_filter_low,highgamma_filter_low]
pain_bands = [delta_filter_high,theta_filter_high,alpha_filter_high,beta_filter_high,gamma_filter_high,highgamma_filter_high]
feature_names= data_h5_high['channels']
pain_band = gamma_filter_low
nopain_band = gamma_filter_high
def combined_function(pain_band, nopain_band):
  plot_features(pain_band,nopain_band,'Before CSP',0, 1,feature_names)
  pain_csp, nopain_csp = csp_cal_appply(pain_band,nopain_band)
  plot_features(pain_csp,nopain_csp,'After CSP',0, 1,feature_names)

combined_function(pain_band, nopain_band)

In [ ]:


# zscored = zscore(a,axis=None)

In [ ]:
a=np.array([[1,2,3,4],[1,2,3,4],[1,2,3,4]])
zscore(a,axis=1)

In [ ]:
data_h5_high.keys()

In [ ]:
data_h5_high['channels'][()]

In [ ]:
indices = [i for i in range(len(data_h5_high['channels'])) if data_h5_high['channels'][i] in [b'RHH8', b'ROF6', b'LHH6', b'RPCC16', b'RHH6', b'LOF6', b'RA12', b'RA7', b'RACC8', b'LACCS6']]


In [ ]:
data_h5_high['channels'][indices]

In [ ]:
indices

In [ ]:
b'LFUSI2' in data_h5_high['channels']

3d plot


In [ ]:
import matplotlib.pyplot as plt
import math
import pdb
# def plot_features(data_high,data_low,choice,ind,chy,feature_names):
#   fig, ax = plt.subplots(figsize=(5,5))
#   # fig, ax = plt.subplots(10, 2, figsize=(15, 30))
#   for channels in ind:
#     # breakpoint()
#     ax.scatter(data_high[channels][::10], data_high[chy][::10], c='b', s=15.0, alpha=0.4, label="Pain data ") #remove the step size
#     ax.scatter(data_low[channels][::10], data_low[chy][::10], c='r', s=15.0, alpha=0.4, label="No Pain data ")#remove the step size

#     ax.grid()
#     ax.set_title(choice)
#     ax.set_xlabel(f'Channel {feature_names[channels]}')
#     ax.set_ylabel(f'Channel {feature_names[chy]}')
#     ax.legend(loc = 0)
def plot_features(data_high, data_low, choice, ind, feature_names):
    fig, axs = plt.subplots(10, 10, figsize=(40, 40))
    for i, channels in enumerate(ind):
        for j, chy in enumerate(ind):
            ax = axs[i, j]
            ax.scatter(data_high[channels][::10], data_high[chy][::10], c='b', s=15.0, alpha=0.4, label="Pain data")
            ax.scatter(data_low[channels][::10], data_low[chy][::10], c='r', s=15.0, alpha=0.4, label="No Pain data")
            ax.grid()
            ax.set_xlabel(f'Channel {feature_names[channels]}')
            ax.set_ylabel(f'Channel {feature_names[chy]}')
            ax.legend()

    plt.tight_layout()
    plt.show()


def csp_cal_appply(data_high, data_low):
  csp=np.array(CSP(data_high.reshape(1,data_high.shape[0],data_high.shape[1]),data_low.reshape(1,data_low.shape[0],data_low.shape[1])))
  pain_csp,nopain_csp = np.transpose(csp[0],(1,0)),np.transpose(csp[1],(1,0))

  pain_data_after_csp = np.matmul(np.transpose(data_high,(1,0)),pain_csp).T
  nopain_data_after_csp = np.matmul(np.transpose(data_low,(1,0)),nopain_csp).T
  return [pain_data_after_csp,nopain_data_after_csp]

In [ ]:
!pip install pdfpages

In [ ]:
from matplotlib.backends.backend_pdf import PdfPages
# from Pdfpages import PdfPages

nopain_bands = [delta_filter_low,theta_filter_low,alpha_filter_low,beta_filter_low,gamma_filter_low,highgamma_filter_low]
nopain_bands_names = ["delta_filter","theta_filter","alpha_filter","beta_filter","gamma_filter","highgamma_filter"]
pain_bands = [delta_filter_high,theta_filter_high,alpha_filter_high,beta_filter_high,gamma_filter_high,highgamma_filter_high]
feature_names= data_h5_high['channels']
pain_band = beta_filter_high
# nopain_band = beta_filter_low
indices = indices = [i for i in range(len(data_h5_high['channels'])) if data_h5_high['channels'][i] in [b'RHH8', b'ROF6', b'LHH6', b'RPCC16', b'RHH6', b'LOF6', b'RA12', b'RA7', b'RACC8', b'LACCS6']]

def combined_function(pain_band, nopain_band,indices):
  for nopain_band in range(len(nopain_bands)):
    print("No pain band = ",nopain_bands_names[nopain_band])
    print("Pain band =beta_filter_high")
    print("Before CSP")
    plot_features(pain_band,nopain_bands[nopain_band],'Before CSP',indices,feature_names)
    # pdf.savefig(fig1)
    pain_csp, nopain_csp = csp_cal_appply(pain_band,nopain_bands[nopain_band])
    print("After CSP")
    plot_features(pain_csp,nopain_csp,'After CSP',indices,feature_names)
    # pdf.savefig(fig2)

combined_function(pain_band, nopain_bands,indices)
# pdf_filename = "axis1.pdf"
# with PdfPages(pdf_filename) as pdf:
#     combined_function(pain_band, nopain_bands, indices, pdf)

In [ ]:
from matplotlib.backends.backend_pdf import PdfPages
# from Pdfpages import PdfPages

nopain_bands = [delta_filter_low,theta_filter_low,alpha_filter_low,beta_filter_low,gamma_filter_low,highgamma_filter_low]
nopain_bands_names = ["delta_filter","theta_filter","alpha_filter","beta_filter","gamma_filter","highgamma_filter"]
pain_bands = [delta_filter_high,theta_filter_high,alpha_filter_high,beta_filter_high,gamma_filter_high,highgamma_filter_high]
feature_names= data_h5_high['channels']
pain_band = beta_filter_high
# nopain_band = beta_filter_low
indices = indices = [i for i in range(len(data_h5_high['channels'])) if data_h5_high['channels'][i] in [b'RHH8', b'ROF6', b'LHH6', b'RPCC16', b'RHH6', b'LOF6', b'RA12', b'RA7', b'RACC8', b'LACCS6']]

def combined_function(pain_band, nopain_band,indices):
  for nopain_band in range(len(nopain_bands)):
    print("No pain band = ",nopain_bands_names[nopain_band])
    print("Pain band =beta_filter_high")
    print("Before CSP")
    plot_features(pain_band,nopain_bands[nopain_band],'Before CSP',indices,feature_names)
    # pdf.savefig(fig1)
    pain_csp, nopain_csp = csp_cal_appply(pain_band,nopain_bands[nopain_band])
    print("After CSP")
    plot_features(pain_csp,nopain_csp,'After CSP',indices,feature_names)
    # pdf.savefig(fig2)

combined_function(pain_band, nopain_bands,indices)
# pdf_filename = "axis1.pdf"
# with PdfPages(pdf_filename) as pdf:
#     combined_function(pain_band, nopain_bands, indices, pdf)

In [ ]:
# Performing PCA
explained_ratio=[]
from sklearn.decomposition import PCA
nopain_bands = [delta_filter_low,theta_filter_low,alpha_filter_low,beta_filter_low,gamma_filter_low,highgamma_filter_low]
pain_bands = [delta_filter_high,theta_filter_high,alpha_filter_high,beta_filter_high,gamma_filter_high,highgamma_filter_high]
n_comp=197
for things in pain_bands+nopain_bands:
  filter = things
  # print(filter.shape)
  pca = PCA(n_components=n_comp)
  pca.fit(filter)
  explained_ratio.append((np.sum(pca.explained_variance_ratio_)))
print(f"Average of the variance explained in all the bands by {n_comp} components is ", np.mean(explained_ratio))






In [ ]:
# np.linalg.matrix_rank(delta_filter_low)
# In the original data, after computing the Instantaneous band power, the rank of the matrix is 197 which means that 1 channel is redundant hence, we can perform PCA to 197 to eliminate the redundancy
#Reducing the number of samples of the channel
from sklearn.decomposition import PCA
nopain_bands_copy = {"delta_filter_low":delta_filter_low.copy(),"theta_filter_low":theta_filter_low.copy(),"alpha_filter_low":alpha_filter_low.copy(),"beta_filter_low":beta_filter_low.copy(),"gamma_filter_low":gamma_filter_low.copy()}
pain_bands_copy = {"delta_filter_high":delta_filter_high.copy(),"theta_filter_high":theta_filter_high.copy(),"alpha_filter_high":alpha_filter_high.copy(),"beta_filter_high":beta_filter_high.copy(),"gamma_filter_high":gamma_filter_high.copy()}
keys_high = list(pain_bands_copy.keys())
keys_low = list(nopain_bands_copy.keys())

for filter_low,filter_high in zip(keys_low,keys_high):
  pca = PCA(n_components=198)
  pain_bands_copy[filter_high] = pca.fit_transform(pain_bands_copy[filter_high])
  nopain_bands_copy[filter_low]=pca.fit_transform(nopain_bands_copy[filter_low])

In [ ]:
#Now eliminating the redundant channel
from sklearn.decomposition import PCA

for filter_low,filter_high in zip(keys_low,keys_high):
  pca = PCA(n_components=197)
  pain_bands_copy[filter_high] = pca.fit_transform(pain_bands_copy[filter_high].T).T
  nopain_bands_copy[filter_low]=pca.fit_transform(nopain_bands_copy[filter_low].T).T


In [ ]:
pain_bands_copy["delta_filter_high"].shape
#channels x #samples

In [ ]:
R = np.matmul(pain_bands_copy["delta_filter_high"],pain_bands_copy["delta_filter_high"].T)

In [ ]:
#ignore this, this is sanity check

pain_bands_copy["delta_filter_high"].shape #n_channelsxn_samples
#ram crashes here
R = np.dot(pain_bands_copy["delta_filter_high"],pain_bands_copy["delta_filter_high"].T)
print(R.shape)
print(np.linalg.matrix_rank(R.T))

In [ ]:
csp = np.array(CSP(pain_bands_copy["delta_filter_high"].reshape(1,pain_bands_copy["delta_filter_high"].shape[0],pain_bands_copy["delta_filter_high"].shape[1]),nopain_bands_copy["delta_filter_low"].reshape(1,nopain_bands_copy["delta_filter_low"].shape[0],nopain_bands_copy["delta_filter_low"].shape[1])))

In [ ]:
csp_pain = csp[0,:,:]
csp_nopain = csp[1,:,:]

In [ ]:
k = 1

# Take the first k rows
first_k_rows = csp_pain[:k, :]

# Take the last k rows
last_k_rows = csp_pain[-k:, :]

# Concatenate the first k rows and last k rows into a single matrix
csp_k_filters = np.concatenate((first_k_rows, last_k_rows), axis=0)

In [ ]:
csp_k_filters.shape

In [ ]:
pain_bands_copy["delta_filter_high"]

In [ ]:
transformed_filter_pain = csp_k_filters @ pain_bands_copy["delta_filter_high"]

In [ ]:
k = 1

# Take the first k rows
first_k_rows = csp_nopain[:k, :]

# Take the last k rows
last_k_rows = csp_nopain[-k:, :]

# Concatenate the first k rows and last k rows into a single matrix
csp_k_filters_np = np.concatenate((first_k_rows, last_k_rows), axis=0)

In [ ]:
transformed_filter_nopain = csp_k_filters_np @ nopain_bands_copy["delta_filter_low"]

In [ ]:
#c=y, cmap=plt.cm.coolwarm, s=50, edgecolors='k'
y = [np.ones((198)),np.zeros((198))]
plt.scatter(transformed_filter_pain[0,:],transformed_filter_pain[1,:],c='red',alpha=0.3)
plt.scatter(transformed_filter_nopain[0,:],transformed_filter_nopain[1,:],c='green',alpha=0.3)

In [ ]:
y = [np.ones((198)),np.zeros((198))]
x=np.arange(len(transformed_filter_pain[0,:]))
plt.scatter(x,transformed_filter_pain[0,:],c='red',alpha=0.3,label='CSP_feature_1')
plt.scatter(x,transformed_filter_pain[1,:],c='blue',alpha=0.3,label='CSP_feature_2')
plt.legend()


In [ ]:
x=np.arange(len(transformed_filter_pain[0,175:]))
# plt.scatter(x,transformed_filter_pain[0,175:],c='red',alpha=0.3,label='CSP_pain_feature_1')
plt.scatter(x,transformed_filter_pain[1,175:],c='blue',alpha=0.3,label='CSP_pain_feature_2')
plt.scatter(x,transformed_filter_nopain[0,175:],c='green',alpha=0.3,label='CSP_nopain_feature_1')
# plt.scatter(x,transformed_filter_nopain[1,175:],c='yellow',alpha=0.3,label='CSP_nopain_feature_2')
plt.legend()


As you can see in the above plot that when we select two csp components of power bands of delta filter signals, only 10-15 samples are distinguishable to a little extent by features 2 of pain state and 1 of no pain state.